In [1]:
import keras
from keras.models import Sequential
from keras import layers
from keras.datasets import mnist
from keras import backend as K

/Users/jeremy.zhang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/jeremy.zhang/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


#### Customized Layer

In [17]:
class antirectifier(layers.Layer):
    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        assert len(shape) == 2  # only valid for 2D tensors
        shape[-1] *= 2
        return (shape[0], shape[1])

    def call(self, inputs):
        inputs -= K.mean(inputs, axis=1, keepdims=True)
        inputs = K.l2_normalize(inputs, axis=1)
        pos = K.relu(inputs)
        neg = K.relu(-inputs)
        return K.concatenate([pos, neg], axis=1)

In [3]:
# global parameters
batch_size = 128
num_classes = 10
epochs = 40

#### Load Data

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 6s 1us/step


In [13]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [9]:
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

In [10]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


#### Model

In [18]:
# build the model
model = Sequential()
model.add(layers.Dense(256, input_shape=(784,)))
model.add(antirectifier())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(256))
model.add(antirectifier())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#### Train

In [19]:
# train the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/40
60000/60000 [==============================] - 4s 70us/step - loss: 0.6671 - acc: 0.9113 - val_loss: 0.2070 - val_acc: 0.9565
Epoch 2/40
60000/60000 [==============================] - 4s 61us/step - loss: 0.1479 - acc: 0.9678 - val_loss: 0.1078 - val_acc: 0.9727
Epoch 3/40
60000/60000 [==============================] - 4s 61us/step - loss: 0.0896 - acc: 0.9779 - val_loss: 0.0867 - val_acc: 0.9762
Epoch 4/40
60000/60000 [==============================] - 4s 64us/step - loss: 0.0660 - acc: 0.9829 - val_loss: 0.0704 - val_acc: 0.9798
Epoch 5/40
60000/60000 [==============================] - 4s 62us/step - loss: 0.0513 - acc: 0.9866 - val_loss: 0.0654 - val_acc: 0.9815
Epoch 6/40
60000/60000 [==============================] - 4s 60us/step - loss: 0.0406 - acc: 0.9893 - val_loss: 0.0684 - val_acc: 0.9803
Epoch 7/40
60000/60000 [==============================] - 4s 61us/step - loss: 0.0348 - acc: 0.9908 - val_loss: 0.0665 - val_acc

#### Compare
compare with an equivalent network
with2x bigger Dense layers and `ReLU`

In [20]:
# build the model
model = Sequential()
model.add(layers.Dense(512, input_shape=(784,), activation="relu"))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/40
60000/60000 [==============================] - 4s 68us/step - loss: 0.2410 - acc: 0.9285 - val_loss: 0.1166 - val_acc: 0.9633
Epoch 2/40
60000/60000 [==============================] - 4s 59us/step - loss: 0.0943 - acc: 0.9715 - val_loss: 0.0759 - val_acc: 0.9770
Epoch 3/40
60000/60000 [==============================] - 4s 61us/step - loss: 0.0627 - acc: 0.9800 - val_loss: 0.0670 - val_acc: 0.9789
Epoch 4/40
60000/60000 [==============================] - 4s 59us/step - loss: 0.0454 - acc: 0.9853 - val_loss: 0.0688 - val_acc: 0.9805
Epoch 5/40
60000/60000 [==============================] - 3s 57us/step - loss: 0.0387 - acc: 0.9872 - val_loss: 0.0693 - val_acc: 0.9781
Epoch 6/40
60000/60000 [==============================] - 3s 57us/step - loss: 0.0311 - acc: 0.9896 - val_loss: 0.0691 - val_acc: 0.9789
Epoch 7/40
60000/60000 [==============================] - 3s 56us/step - loss: 0.0251 - acc: 0.9913 - val_loss: 0.0709 - val_acc